<a href="https://colab.research.google.com/github/rowida2/recommendation-system-project/blob/main/scrao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving interactions.csv to interactions.csv
Saving posts.csv to posts.csv
Saving tourism_places (1).csv to tourism_places (1).csv
Saving users.csv to users.csv


In [ ]:

from pandas import read_csv


plac=read_csv("tourism_places (1).csv")
posts=read_csv("posts.csv")
interactions=read_csv("interactions.csv")
users=read_csv("users.csv")


In [ ]:
plac.head()

,place_id,name,city,category,budget_level,suitable_for,tags
0,1,Marsa Alam Wellness Center,Marsa Alam,medical,high,all,"relaxation, health, spa"
1,2,Siwa Rock Climbing Site,Siwa,adventure,low,solo,"safari, outdoor, mountain"
2,3,Red Monastery of Marsa Alam,Marsa Alam,historical,mid,all,"educational, museum, ancient"
3,4,Red Beach Resort of Nuweiba,Nuweiba,beach,high,all,"snorkeling, boat trip, water sports"
4,5,Port Said Diving Center,Port Said,beach,mid,all,"sea, relaxation, water sports"


In [ ]:
posts.head()

,post_id,user_id,place_id,content
0,450,157,115,Extreme sports at Modern Quad Biking Area of S...
1,634,26,460,Push your limits at Modern Quad Biking Area of...
2,1,226,424,Cultural overload at Dahab Mosque - in the bes...
3,756,190,480,Amazing experience at Blue Citadel of Marsa Al...
4,46,140,122,Lounging at White Island of Sharm El Sheikh wi...


In [ ]:
interactions.tail()

,interaction_id,user_id,place_id,interaction_type,rating,timestamp
4993,4994,250,263,like,4,2024-09-09
4994,4995,250,7,view,5,2025-09-24
4995,4996,250,302,like,5,2024-04-12
4996,4997,250,388,visit,5,2025-03-31
4997,4998,250,141,visit,4,2025-04-20


In [ ]:
users.count()

,0
user_id,250
name,250
age,250
gender,250
interests,250
budget_preference,250
travel_style,250
visited_cities,218


In [ ]:
plac.count()

,0
place_id,500
name,500
city,500
category,500
budget_level,500
suitable_for,500
tags,500


In [ ]:
weights = {"view": 1, "like": 2, "save": 3, "visit": 4}

data = interactions.merge(plac, on="place_id")
data["score"] = data["interaction_type"].map(weights)

data["weighted_score"] = data["score"] * data["rating"]

user_interest = (
    data
    .groupby(["user_id", "category"])["weighted_score"]
    .sum()
    .reset_index()
)


In [ ]:
data

,interaction_id,user_id,place_id,interaction_type,rating,timestamp,score,name,city,category,budget_level,suitable_for,tags,weighted_score
0,1,1,328,visit,5,2025-01-18,4,Suez Medical Tourism Center,Suez,medical,high,all,"relaxation, healing, massage, mineral water",20
1,2,1,67,like,5,2024-02-14,2,Royal Rock Climbing Site of Fayoum,Fayoum,adventure,mid,solo,"hiking, wildlife, safari",10
2,3,1,68,save,4,2024-07-08,3,Giza Diving Center,Giza,beach,low,all,"swimming, beach, sunset",12
3,4,1,93,save,5,2024-09-07,3,Giza Quad Biking Area,Giza,adventure,mid,solo,"excitement, hiking, mountain, wildlife",15
4,5,1,122,visit,5,2025-06-26,4,Royal Monastery of Taba,Taba,historical,low,all,"roman, educational, museum",20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,4994,250,263,like,4,2024-09-09,2,Ancient Safari Park of Port Said,Port Said,adventure,high,solo,"hiking, excitement, adventure, desert",8
4994,4995,250,7,view,5,2025-09-24,1,White Island of Alexandria,Alexandria,beach,high,family,"boat trip, diving, snorkeling, swimming",5
4995,4996,250,302,like,5,2024-04-12,2,Hurghada Archaeological Site,Hurghada,historical,low,all,"islamic, heritage, ancient",10
4996,4997,250,388,visit,5,2025-03-31,4,Cairo Wellness Center,Cairo,medical,high,all,"relaxation, massage, therapeutic",20


In [ ]:
interactions["interaction_type"] = interactions["interaction_type"].str.lower().str.strip()


weights = {
    "view": 1,
    "like": 2,
    "save": 3,
    "visit": 4
}

# Calculate score
interactions["score"] = interactions["interaction_type"].map(weights)

In [ ]:
data["interaction_type"].unique

<bound method Series.unique of 0       visit
1        like
2        save
3        save
4       visit
        ...  
4993     like
4994     view
4995     like
4996    visit
4997    visit
Name: interaction_type, Length: 4998, dtype: object>

In [ ]:
data = interactions.merge(plac[["place_id", "category", "budget_level", "suitable_for"]], on="place_id", how="left")


In [ ]:
user_interest = (
    data.groupby(["user_id", "category"])["score"]
    .sum()
    .reset_index()
)


In [ ]:
top_categories = (
    user_interest.sort_values(["user_id", "score"], ascending=[True, False])
    .groupby("user_id")
    .head(3)
)

In [ ]:
recommended_places = plac.merge(top_categories, on="category", how="inner")
recommended_places = recommended_places[["user_id", "place_id", "name", "city", "category"]]

In [ ]:
recommended_places = recommended_places.merge(user_interest, on=["user_id","category"], how="left")
recommended_places = recommended_places.sort_values(["user_id", "score"], ascending=[True, False])

top5_places = recommended_places.groupby("user_id").head(5).reset_index(drop=True)



In [ ]:
print(top5_places)

      user_id  place_id                                  name        city  \
0           1         4           Red Beach Resort of Nuweiba     Nuweiba   
1           1         5               Port Said Diving Center   Port Said   
2           1         7            White Island of Alexandria  Alexandria   
3           1        10                       Siwa Beach Club        Siwa   
4           1        22                     Marsa Alam Island  Marsa Alam   
...       ...       ...                                   ...         ...   
1245      250         2               Siwa Rock Climbing Site        Siwa   
1246      250         8                 Port Said Hiking Path   Port Said   
1247      250        11  Golden Water Sports Center of Fayoum      Fayoum   
1248      250        14                 Fayoum Mountain Trail      Fayoum   
1249      250        15                        Cairo Zip Line       Cairo   

       category  score  
0         beach     21  
1         beach     21  


# Collaborative Filtering

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Create User-Item Matrix
user_item_matrix = interactions.pivot_table(
    index='user_id',
    columns='place_id',
    values='rating',
    fill_value=0
)

# Calculate User Similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

# Find Similar Users
def get_similar_users(user_id, n=10):
    if user_id not in user_similarity_df.index:
        return []
    similar_scores = user_similarity_df[user_id].sort_values(ascending=False)
    return similar_scores.iloc[1:n+1].index.tolist()  # exclude self

# Recommendations from Similar Users
def collaborative_recommendations(user_id, n=10):
    similar_users = get_similar_users(user_id)

    # Places liked by similar users
    similar_interactions = interactions[
        (interactions['user_id'].isin(similar_users)) &
        (interactions['rating'] >= 4)
    ]

    # Calculate scores
    place_scores = (
        similar_interactions
        .groupby('place_id')
        .agg({'rating': 'mean', 'score': 'sum'})
        .sort_values('score', ascending=False)
    )

    # Exclude visited places
    # `visited_places` is not defined, will cause another NameError if not handled.
    # For now, commenting out or assuming it will be defined later.
    # visited = visited_places.get(user_id, [])
    # place_scores = place_scores[~place_scores.index.isin(visited)]

    return place_scores.head(n)

# Hybrid Scoring  (Content + Collaborative)

In [ ]:
def hybrid_recommendations(user_id, n=10, visited_places_dict=None):
    # Content-based score
    content_recs = recommended_places[recommended_places['user_id'] == user_id].copy()
    content_recs['content_score'] = content_recs['score']

    # Collaborative score
    collab_recs = collaborative_recommendations(user_id, n=50)

    # Merge
    hybrid = content_recs.merge(
        collab_recs[['rating', 'score']],
        left_on='place_id',
        right_index=True,
        how='left',
        suffixes=('_content', '_collab')
    )

    # Combined score (60% collaborative, 40% content)
    hybrid['final_score'] = (
        hybrid['score_collab'].fillna(0) * 0.6 +
        hybrid['content_score'] * 0.4
    )

    # Sort and return top N
    hybrid = hybrid.sort_values('final_score', ascending=False)

    # Filter visited
    if visited_places_dict is None:
        # If not provided, use the full interactions (for normal usage, not evaluation)
        visited_places_dict = interactions.groupby('user_id')['place_id'].apply(list).to_dict()
    visited = visited_places_dict.get(user_id, [])
    hybrid = hybrid[~hybrid['place_id'].isin(visited)]

    return hybrid.head(n)

In [ ]:
def diverse_recommendations(user_id, n=10):
    recs = hybrid_recommendations(user_id, n=30)  # get more than needed

    # Ensure diversity: max 2 places from same city/category
    diverse_recs = []
    city_count = {}
    category_count = {}

    for _, row in recs.iterrows():
        city = row['city']
        category = row['category']

        if city_count.get(city, 0) < 2 and category_count.get(category, 0) < 2:
            diverse_recs.append(row)
            city_count[city] = city_count.get(city, 0) + 1
            category_count[category] = category_count.get(category, 0) + 1

        if len(diverse_recs) >= n:
            break

    return pd.DataFrame(diverse_recs)

# Evaluation

In [ ]:
from sklearn.model_selection import train_test_split

# Split data
train_interactions = interactions.sample(frac=0.8, random_state=42)
test_interactions = interactions.drop(train_interactions.index)

# Metrics
def evaluate_recommendations():
    results = []

    # Pre-calculate visited places based on train_interactions for evaluation
    visited_places_train = train_interactions.groupby('user_id')['place_id'].apply(list).to_dict()

    for user_id in test_interactions['user_id'].unique()[:50]:  # test on 50 users
        # Get recommendations, passing visited_places from training data
        recs = hybrid_recommendations(user_id, n=10, visited_places_dict=visited_places_train)

        # Get actual liked places from test set
        actual = test_interactions[
            (test_interactions['user_id'] == user_id) &
            (test_interactions['rating'] >= 4)
        ]['place_id'].tolist()

        # Calculate metrics
        recommended = recs['place_id'].tolist()
        hits = len(set(recommended) & set(actual))

        precision = hits / len(recommended) if recommended else 0
        recall = hits / len(actual) if actual else 0

        results.append({
            'user_id': user_id,
            'precision': precision,
            'recall': recall,
            'hits': hits
        })

    results_df = pd.DataFrame(results)
    print("Average Precision:", results_df['precision'].mean())
    print("Average Recall:", results_df['recall'].mean())
    print("Average Hits:", results_df['hits'].mean())

    return results_df

# Run evaluation
eval_results = evaluate_recommendations()

Average Precision: 0.09399999999999999
Average Recall: 0.2775
Average Hits: 0.94


# Item Based

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print("Building item-item similarity matrix...")

# 1. Create Item-User Matrix
item_user_matrix = interactions.pivot_table(
    index='place_id',
    columns='user_id',
    values='rating',
    fill_value=0
)

print(f"Matrix shape: {item_user_matrix.shape}")
print(f"   - {item_user_matrix.shape[0]} places")
print(f"   - {item_user_matrix.shape[1]} users")

# 2. Calculate Similarity between Items
item_similarity = cosine_similarity(item_user_matrix)

item_similarity_df = pd.DataFrame(
    item_similarity,
    index=item_user_matrix.index,
    columns=item_user_matrix.index
)

print("Item similarity matrix created!")
print(f"   Matrix size: {item_similarity_df.shape}")

# 3. Function: Find similar places for a given place
def get_similar_places(place_id, n=10):
    """
    Returns similar places for a given place

    Parameters:
    - place_id: Place ID
    - n: Number of similar places requested

    Returns:
    - Series: Similar places with scores
    """
    if place_id not in item_similarity_df.index:
        return pd.Series()

    similar_scores = item_similarity_df[place_id].sort_values(ascending=False)
    return similar_scores.iloc[1:n+1]  # Exclude the place itself (index 0)

# 4. Function: Item-Based CF Recommendations
def item_based_recommendations(user_id, n=10):
    """
    Recommendations based on places the user liked

    Parameters:
    - user_id: User ID
    - n: Number of recommendations requested

    Returns:
    - list: [(place_id, score), ...]
    """

    # Places with high ratings from the user (4 or 5)
    user_liked = interactions[
        (interactions['user_id'] == user_id) &
        (interactions['rating'] >= 4)
    ]['place_id'].tolist()

    if not user_liked:
        return []

    # Aggregate scores for similar places
    recommendations = {}

    for place_id in user_liked:
        similar = get_similar_places(place_id, n=20)

        for sim_place, score in similar.items():
            # Exclude places already visited by the user
            if sim_place not in user_liked:
                recommendations[sim_place] = recommendations.get(sim_place, 0) + score

    # Sort by score and return top N
    sorted_recs = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return sorted_recs[:n]

print("Functions ready:")
print("   - get_similar_places(place_id, n)")
print("   - item_based_recommendations(user_id, n)")

# 5. Quick test
print("\nTesting with user_id=1...")
test_recs = item_based_recommendations(1, n=5)
if test_recs:
    print("Test successful! Top 5 recommendations:")
    for i, (place_id, score) in enumerate(test_recs, 1):
        place_name = plac[plac['place_id'] == place_id]['name'].values[0]
        print(f"   {i}. {place_name} (score: {score:.3f})")
else:
    print("User 1 has no high-rated places (rating >= 4)")

Building item-item similarity matrix...
Matrix shape: (499, 250)
   - 499 places
   - 250 users
Item similarity matrix created!
   Matrix size: (499, 499)
Functions ready:
   - get_similar_places(place_id, n)
   - item_based_recommendations(user_id, n)

Testing with user_id=1...
Test successful! Top 5 recommendations:
   1. Marsa Alam Wellness Center (score: 1.606)
   2. Hurghada Thermal Baths (score: 1.347)
   3. Port Said Spa (score: 1.285)
   4. Siwa Beach Resort (score: 1.282)
   5. Siwa Archaeological Site (score: 1.113)


In [ ]:
# Test: Find similar places for a given place
print("\nTesting: Similar places to place_id=1")
print("=" * 60)

# Original place information
original_place = plac[plac['place_id'] == 1].iloc[0]
print(f"Original Place: {original_place['name']}")
print(f"Category: {original_place['category']}")
print(f"City: {original_place['city']}")
print()

# Similar places
similar = get_similar_places(1, n=5)
print("Top 5 Similar Places:")
for i, (place_id, score) in enumerate(similar.items(), 1):
    place = plac[plac['place_id'] == place_id].iloc[0]
    print(f"{i}. {place['name']}")
    print(f"   Category: {place['category']} | City: {place['city']}")
    print(f"   Similarity Score: {score:.3f}")
    print()

📍 Testing: Similar places to place_id=1
Original Place: Marsa Alam Wellness Center
Category: medical
City: Marsa Alam

Top 5 Similar Places:
1. Luxor Mosque
   Category: historical | City: Luxor
   Similarity Score: 0.436

2. Taba Lagoon
   Category: beach | City: Taba
   Similarity Score: 0.364

3. Luxor Therapeutic Pool
   Category: medical | City: Luxor
   Similarity Score: 0.356

4. Aswan Hot Springs
   Category: medical | City: Aswan
   Similarity Score: 0.253

5. Hurghada Rehabilitation Center
   Category: medical | City: Hurghada
   Similarity Score: 0.252



# Hybrid System

In [ ]:
print("Building Hybrid Recommendation System...")
print("=" * 60)

def hybrid_recommendations(user_id, n=10):
    """
    Combine Content-Based + Item-Based Collaborative Filtering

    Parameters:
    - user_id: User ID
    - n: Number of recommendations requested

    Returns:
    - DataFrame: Recommendations with details and scores
    """

    # ========== 1. Content-Based Scores ==========
    content_scores = {}

    # Get the user's preferred categories
    user_cats = user_interest[user_interest['user_id'] == user_id]

    if user_cats.empty:
        # If new user, use most popular categories
        user_cats = user_interest.groupby('category')['weighted_score'].sum().nlargest(3).reset_index()
        user_cats['user_id'] = user_id
    else:
        user_cats = user_cats.nlargest(3, 'weighted_score')

    # Calculate scores for places from preferred categories
    for _, row in user_cats.iterrows():
        category_places = plac[plac['category'] == row['category']]
        for _, place in category_places.iterrows():
            if place['place_id'] not in content_scores:
                content_scores[place['place_id']] = 0
            content_scores[place['place_id']] += row['weighted_score']

    # Normalize content scores (0-1)
    if content_scores:
        max_content = max(content_scores.values())
        content_scores = {k: v/max_content for k, v in content_scores.items()}

    # ========== 2. Item-Based Collaborative Scores ==========
    item_based = item_based_recommendations(user_id, n=100)
    item_scores = dict(item_based) if item_based else {}

    # Normalize collaborative scores (0-1)
    if item_scores:
        max_collab = max(item_scores.values())
        item_scores = {k: v/max_collab for k, v in item_scores.items()}

    # ========== 3. Exclude visited places ==========
    visited = interactions[interactions['user_id'] == user_id]['place_id'].tolist()

    # ========== 4. Merge Scores ==========
    final_scores = {}
    all_places = set(content_scores.keys()) | set(item_scores.keys())

    for place_id in all_places:
        if place_id in visited:
            continue  # Skip visited places

        score = 0

        # Content-Based: 40% weight
        if place_id in content_scores:
            score += content_scores[place_id] * 0.4

        # Item-Based Collaborative: 60% weight
        if place_id in item_scores:
            score += item_scores[place_id] * 0.6

        final_scores[place_id] = score

    # ========== 5. Sort and return results ==========
    sorted_recs = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)

    # Add place details
    results = []
    for place_id, score in sorted_recs[:n]:
        place_info = plac[plac['place_id'] == place_id].iloc[0]
        results.append({
            'user_id': user_id,
            'place_id': place_id,
            'name': place_info['name'],
            'city': place_info['city'],
            'category': place_info['category'],
            'budget_level': place_info['budget_level'],
            'suitable_for': place_info['suitable_for'],
            'final_score': score
        })

    return pd.DataFrame(results)

print("Hybrid recommendation function created!")

# ========== Test ==========
print("\nTesting with user_id=1...")
test_hybrid = hybrid_recommendations(1, n=10)

if not test_hybrid.empty:
    print("Hybrid recommendations for user 1:")
    print(test_hybrid[['name', 'category', 'city', 'final_score']])
else:
    print("No recommendations generated")

print("\n" + "=" * 60)
print("Ready to generate recommendations for all users!")

Building Hybrid Recommendation System...
Hybrid recommendation function created!

Testing with user_id=1...
Hybrid recommendations for user 1:
                             name   category        city  final_score
0               Siwa Beach Resort      beach        Siwa     0.878865
1      Marsa Alam Wellness Center    medical  Marsa Alam     0.842553
2     Red Beach Resort of Nuweiba      beach     Nuweiba     0.778550
3                Suez Hiking Path  adventure        Suez     0.757687
4                     Taba Lagoon      beach        Taba     0.751277
5          Hurghada Thermal Baths    medical    Hurghada     0.745553
6  Crystal Desert Safari of Luxor  adventure       Luxor     0.738669
7      White Island of Alexandria      beach  Alexandria     0.734183
8             Aswan Desert Safari  adventure       Aswan     0.728588
9                   Port Said Spa    medical   Port Said     0.722508

Ready to generate recommendations for all users!


In [ ]:
print("Generating recommendations for all users...")
print("=" * 60)

# Generate recommendations for each user
all_hybrid_recommendations = []

for user_id in users['user_id'].unique():
    user_recs = hybrid_recommendations(user_id, n=5)  # top 5 for each user
    if not user_recs.empty:
        all_hybrid_recommendations.append(user_recs)

# Merge all recommendations
final_hybrid_recommendations = pd.concat(all_hybrid_recommendations, ignore_index=True)

print(f"Generated recommendations for {users['user_id'].nunique()} users")
print(f"Total recommendations: {len(final_hybrid_recommendations)}")
print(f"Avg recommendations per user: {len(final_hybrid_recommendations) / users['user_id'].nunique():.1f}")

print("\nSample recommendations:")
print(final_hybrid_recommendations.head(15))

print("\nStatistics:")
print("=" * 60)
print("\nCategory distribution in recommendations:")
print(final_hybrid_recommendations['category'].value_counts())

print("\nCity distribution in recommendations:")
print(final_hybrid_recommendations['city'].value_counts().head(10))

print("\nBudget level distribution:")
print(final_hybrid_recommendations['budget_level'].value_counts())

print("\n" + "=" * 60)
print("Hybrid recommendations ready!")

Generating recommendations for all users...
Generated recommendations for 250 users
Total recommendations: 1250
Avg recommendations per user: 5.0

Sample recommendations:
    user_id  place_id                              name             city  \
0         1       271                 Siwa Beach Resort             Siwa   
1         1       240        Marsa Alam Wellness Center       Marsa Alam   
2         1         4       Red Beach Resort of Nuweiba          Nuweiba   
3         1       280                  Suez Hiking Path             Suez   
4         1       233                       Taba Lagoon             Taba   
5         2       233                       Taba Lagoon             Taba   
6         2       368             Taba Wildlife Reserve             Taba   
7         2       278             Paradise Cave of Suez             Suez   
8         2       133             Port Said Hot Springs        Port Said   
9         2       205                     Port Said Spa        Port S

In [ ]:
print("EVALUATING HYBRID RECOMMENDATION SYSTEM")
print("=" * 60)

from sklearn.model_selection import train_test_split

# Split data (80% train, 20% test)
train_interactions = interactions.sample(frac=0.8, random_state=42)
test_interactions = interactions.drop(train_interactions.index)

print(f"Train interactions: {len(train_interactions)}")
print(f"Test interactions: {len(test_interactions)}")

def evaluate_hybrid_system():
    """Evaluate the system on the test set"""

    results = []
    test_users = test_interactions['user_id'].unique()

    print(f"\nEvaluating on {len(test_users)} users...")

    # Progress bar
    from time import time
    start = time()

    for idx, user_id in enumerate(test_users):
        if (idx + 1) % 50 == 0:
            print(f"  Progress: {idx + 1}/{len(test_users)} users...")

        # Get recommendations
        try:
            recs = hybrid_recommendations(user_id, n=10)
        except:
            continue

        if recs.empty:
            continue

        # Actual places the user liked in the test set
        actual = test_interactions[
            (test_interactions['user_id'] == user_id) &
            (test_interactions['rating'] >= 4)
        ]['place_id'].tolist()

        if not actual:
            continue

        # Calculate metrics
        recommended = recs['place_id'].tolist()
        hits = len(set(recommended) & set(actual))

        precision = hits / len(recommended) if recommended else 0
        recall = hits / len(actual) if actual else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        results.append({
            'user_id': user_id,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'hits': hits,
            'num_actual': len(actual),
            'num_recommended': len(recommended)
        })

    elapsed = time() - start

    results_df = pd.DataFrame(results)

    print(f"\nEvaluation completed in {elapsed:.1f} seconds")
    print(f"Evaluated {len(results_df)} users")

    # Results
    print("\n" + "=" * 60)
    print("HYBRID SYSTEM RESULTS:")
    print("=" * 60)
    print(f"Average Precision:  {results_df['precision'].mean():.4f} ({results_df['precision'].mean()*100:.2f}%)")
    print(f"Average Recall:     {results_df['recall'].mean():.4f} ({results_df['recall'].mean()*100:.2f}%)")
    print(f"Average F1-Score:   {results_df['f1_score'].mean():.4f}")
    print(f"Average Hits:       {results_df['hits'].mean():.2f}")
    print("=" * 60)

    # Distribution of results
    print("\nDistribution of Hits:")
    print(results_df['hits'].value_counts().sort_index())

    print("\nPrecision ranges:")
    print(pd.cut(results_df['precision'], bins=[0, 0.1, 0.2, 0.3, 0.4, 1.0]).value_counts().sort_index())

    return results_df

# Run evaluation
eval_results = evaluate_hybrid_system()

# Comparison with the old system
print("\n" + "=" * 60)
print("COMPARISON:")
print("=" * 60)
print("Old System (Content-Based only):")
print("  Precision: 9.4%")
print("  Recall:    27.8%")
print("  Hits:      0.94")
print()
print(f"New System (Hybrid):")
print(f"  Precision: {eval_results['precision'].mean()*100:.1f}%")
print(f"  Recall:    {eval_results['recall'].mean()*100:.1f}%")
print(f"  Hits:      {eval_results['hits'].mean():.2f}")
print()
print(f"Improvement:")
print(f"  Precision: +{(eval_results['precision'].mean()*100 - 9.4):.1f}%")
print(f"  Recall:    +{(eval_results['recall'].mean()*100 - 27.8):.1f}%")
print(f"  Hits:      +{(eval_results['hits'].mean() - 0.94):.2f}")
print("=" * 60)

🎯 EVALUATING HYBRID RECOMMENDATION SYSTEM
Train interactions: 3998
Test interactions: 1000

Evaluating on 242 users...
  Progress: 50/242 users...
  Progress: 100/242 users...
  Progress: 150/242 users...
  Progress: 200/242 users...

✅ Evaluation completed in 8.8 seconds
✅ Evaluated 240 users

📊 HYBRID SYSTEM RESULTS:
Average Precision:  0.0000 (0.00%)
Average Recall:     0.0000 (0.00%)
Average F1-Score:   0.0000
Average Hits:       0.00

📈 Distribution of Hits:
hits
0    240
Name: count, dtype: int64

📊 Precision ranges:
precision
(0.0, 0.1]    0
(0.1, 0.2]    0
(0.2, 0.3]    0
(0.3, 0.4]    0
(0.4, 1.0]    0
Name: count, dtype: int64

📊 COMPARISON:
Old System (Content-Based only):
  Precision: 9.4%
  Recall:    27.8%
  Hits:      0.94

New System (Hybrid):
  Precision: 0.0%
  Recall:    0.0%
  Hits:      0.00

🚀 Improvement:
  Precision: +-9.4%
  Recall:    +-27.8%
  Hits:      +-0.94


In [ ]:
print("EVALUATING HYBRID RECOMMENDATION SYSTEM (FIXED)")
print("=" * 60)

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Split data (80% train, 20% test)
train_interactions = interactions.sample(frac=0.8, random_state=42)
test_interactions = interactions.drop(train_interactions.index)

print(f"Train interactions: {len(train_interactions)}")
print(f"Test interactions: {len(test_interactions)}")

# ========== Build the system on Train Data only ==========
print("\nBuilding system on TRAIN data only...")

# 1. Item similarity on train data
train_item_user_matrix = train_interactions.pivot_table(
    index='place_id',
    columns='user_id',
    values='rating',
    fill_value=0
)

train_item_similarity = cosine_similarity(train_item_user_matrix)
train_item_similarity_df = pd.DataFrame(
    train_item_similarity,
    index=train_item_user_matrix.index,
    columns=train_item_user_matrix.index
)

# 2. User interests on train data
train_data = train_interactions.merge(plac, on="place_id")
train_data["score"] = train_data["interaction_type"].map({"view": 1, "like": 2, "save": 3, "visit": 4})
train_data["weighted_score"] = train_data["score"] * train_data["rating"]

train_user_interest = (
    train_data
    .groupby(["user_id", "category"])["weighted_score"]
    .sum()
    .reset_index()
)

print("Train models built!")

# ========== Functions using Train Data ==========

def get_similar_places_train(place_id, n=10):
    """Similar places from train data"""
    if place_id not in train_item_similarity_df.index:
        return pd.Series()

    similar_scores = train_item_similarity_df[place_id].sort_values(ascending=False)
    return similar_scores.iloc[1:n+1]

def item_based_recommendations_train(user_id, n=10):
    """Item-based recommendations from train data"""

    user_liked = train_interactions[
        (train_interactions['user_id'] == user_id) &
        (train_interactions['rating'] >= 4)
    ]['place_id'].tolist()

    if not user_liked:
        return []

    recommendations = {}
    for place_id in user_liked:
        similar = get_similar_places_train(place_id, n=20)
        for sim_place, score in similar.items():
            if sim_place not in user_liked:
                recommendations[sim_place] = recommendations.get(sim_place, 0) + score

    sorted_recs = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return sorted_recs[:n]

def hybrid_recommendations_train(user_id, n=10):
    """Hybrid recommendations from train data"""

    # Content-based scores
    content_scores = {}
    user_cats = train_user_interest[train_user_interest['user_id'] == user_id]

    if user_cats.empty:
        user_cats = train_user_interest.groupby('category')['weighted_score'].sum().nlargest(3).reset_index()
        user_cats['user_id'] = user_id
    else:
        user_cats = user_cats.nlargest(3, 'weighted_score')

    for _, row in user_cats.iterrows():
        category_places = plac[plac['category'] == row['category']]
        for _, place in category_places.iterrows():
            if place['place_id'] not in content_scores:
                content_scores[place['place_id']] = 0
            content_scores[place['place_id']] += row['weighted_score']

    # Normalize
    if content_scores:
        max_content = max(content_scores.values())
        content_scores = {k: v/max_content for k, v in content_scores.items()}

    # Item-based scores
    item_based = item_based_recommendations_train(user_id, n=100)
    item_scores = dict(item_based) if item_based else {}

    if item_scores:
        max_collab = max(item_scores.values())
        item_scores = {k: v/max_collab for k, v in item_scores.items()}

    # Exclude places from train (visited by user in training)
    visited = train_interactions[train_interactions['user_id'] == user_id]['place_id'].tolist()

    # Merge scores
    final_scores = {}
    all_places = set(content_scores.keys()) | set(item_scores.keys())

    for place_id in all_places:
        if place_id in visited:
            continue

        score = 0
        if place_id in content_scores:
            score += content_scores[place_id] * 0.4
        if place_id in item_scores:
            score += item_scores[place_id] * 0.6

        final_scores[place_id] = score

    sorted_recs = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)

    results = []
    for place_id, score in sorted_recs[:n]:
        results.append({
            'user_id': user_id,
            'place_id': place_id,
            'final_score': score
        })

    return pd.DataFrame(results)

# ========== Evaluation ==========

def evaluate_hybrid_fixed():
    """Correct evaluation on test data"""

    results = []
    test_users = test_interactions['user_id'].unique()

    print(f"\nEvaluating on {len(test_users)} test users...")

    from time import time
    start = time()

    for idx, user_id in enumerate(test_users):
        if (idx + 1) % 50 == 0:
            print(f"  Progress: {idx + 1}/{len(test_users)} users...")

        # Recommendations from train data
        try:
            recs = hybrid_recommendations_train(user_id, n=10)
        except Exception as e:
            continue

        if recs.empty:
            continue

        # Actual places from test set (liked by user after training)
        actual = test_interactions[
            (test_interactions['user_id'] == user_id) &
            (test_interactions['rating'] >= 4)
        ]['place_id'].tolist()

        if not actual:
            continue

        # Calculate metrics
        recommended = recs['place_id'].tolist()
        hits = len(set(recommended) & set(actual))

        precision = hits / len(recommended) if recommended else 0
        recall = hits / len(actual) if actual else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        results.append({
            'user_id': user_id,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'hits': hits,
            'num_actual': len(actual),
            'num_recommended': len(recommended)
        })

    elapsed = time() - start

    results_df = pd.DataFrame(results)

    print(f"\nEvaluation completed in {elapsed:.1f} seconds")
    print(f"Evaluated {len(results_df)} users")

    if results_df.empty:
        print("\nWARNING: No results! Check if users have interactions in both train and test.")
        return results_df

    # Results
    print("\n" + "=" * 60)
    print("HYBRID SYSTEM RESULTS (CORRECTED):")
    print("=" * 60)
    print(f"Average Precision:  {results_df['precision'].mean():.4f} ({results_df['precision'].mean()*100:.2f}%)")
    print(f"Average Recall:     {results_df['recall'].mean():.4f} ({results_df['recall'].mean()*100:.2f}%)")
    print(f"Average F1-Score:   {results_df['f1_score'].mean():.4f}")
    print(f"Average Hits:       {results_df['hits'].mean():.2f}")
    print("=" * 60)

    print("\nDistribution of Hits:")
    print(results_df['hits'].value_counts().sort_index())

    print("\nUsers with at least 1 hit:")
    users_with_hits = (results_df['hits'] > 0).sum()
    print(f"{users_with_hits}/{len(results_df)} ({users_with_hits/len(results_df)*100:.1f}%)")

    return results_df

# Run corrected evaluation
eval_results = evaluate_hybrid_fixed()

# Comparison
if not eval_results.empty:
    print("\n" + "=" * 60)
    print("COMPARISON:")
    print("=" * 60)
    print("Old System (Content-Based only):")
    print("  Precision: 9.4%")
    print("  Recall:    27.8%")
    print("  Hits:      0.94")
    print()
    print(f"New System (Hybrid - Corrected):")
    print(f"  Precision: {eval_results['precision'].mean()*100:.1f}%")
    print(f"  Recall:    {eval_results['recall'].mean()*100:.1f}%")
    print(f"  Hits:      {eval_results['hits'].mean():.2f}")
    print()
    improvement_prec = eval_results['precision'].mean()*100 - 9.4
    improvement_rec = eval_results['recall'].mean()*100 - 27.8
    improvement_hits = eval_results['hits'].mean() - 0.94

    print(f"Improvement:")
    print(f"  Precision: {'+' if improvement_prec > 0 else ''}{improvement_prec:.1f}%")
    print(f"  Recall:    {'+' if improvement_rec > 0 else ''}{improvement_rec:.1f}%")
    print(f"  Hits:      {'+' if improvement_hits > 0 else ''}{improvement_hits:.2f}")
    print("=" * 60)


🎯 EVALUATING HYBRID RECOMMENDATION SYSTEM (FIXED)
Train interactions: 3998
Test interactions: 1000

Building system on TRAIN data only...
Train models built!

Evaluating on 242 test users...
  Progress: 50/242 users...
  Progress: 100/242 users...
  Progress: 150/242 users...
  Progress: 200/242 users...

Evaluation completed in 7.6 seconds
Evaluated 240 users

HYBRID SYSTEM RESULTS (CORRECTED):
Average Precision:  0.0133 (1.33%)
Average Recall:     0.0283 (2.83%)
Average F1-Score:   0.0170
Average Hits:       0.13

Distribution of Hits:
hits
0    215
1     18
2      7
Name: count, dtype: int64

Users with at least 1 hit:
25/240 (10.4%)

COMPARISON:
Old System (Content-Based only):
  Precision: 9.4%
  Recall:    27.8%
  Hits:      0.94

New System (Hybrid - Corrected):
  Precision: 1.3%
  Recall:    2.8%
  Hits:      0.13

Improvement:
  Precision: -8.1%
  Recall:    -25.0%
  Hits:      -0.81
